In [8]:
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
import psutil
import plotly.graph_objects as go
import chart_studio
import chart_studio.plotly as py

import plotly.io as pio

chart_studio.tools.set_config_file(world_readable=True,
                             sharing='public')


In [9]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
stonks = pd.read_csv("stonks.csv")

In [11]:
stonks = stonks.iloc[::-1]
stonks.head()

,timestamp,open,high,low,close,volume
5032,2000-03-29,151.5625,152.4843,149.6562,151.2187,6747500
5031,2000-03-30,150.1562,151.9218,147.1250,148.6875,9491900
5030,2000-03-31,149.6250,152.3125,148.4375,150.3750,9249100
5029,2000-04-03,150.1250,151.2500,148.6875,151.2500,8508200
5028,2000-04-04,151.7500,153.0000,141.3906,150.1250,19585500


In [12]:
split_percent = 0.7
val_percent = 0.1

close_data = stonks[["open", "high", "low", "close"]].values.reshape((-1,4))

split = int(split_percent*len(close_data))
val = int((val_percent + split_percent) * len(close_data))
close_train = close_data[:split]
close_val = close_data[split:val]
close_test = close_data[val:]

date_train = stonks.timestamp[:split]
date_val = stonks.timestamp[split:val]
date_test = stonks.timestamp[val:]

close_train[0:, 3:]

array([[151.2187],
       [148.6875],
       [150.375 ],
       ...,
       [185.49  ],
       [187.01  ],
       [188.25  ]])

In [35]:
look_back = 15
num_epochs = 64

train_generator = keras.preprocessing.sequence.TimeseriesGenerator(close_train, close_train, length=look_back, batch_size=20)   
val_generator = keras.preprocessing.sequence.TimeseriesGenerator(close_val, close_val, length=look_back, batch_size=10)     
test_generator = keras.preprocessing.sequence.TimeseriesGenerator(close_test, close_test, length=look_back, batch_size=1)


def customloss():
    def loss(y_true, y_pred):
        # Use x here as you wish
        err = keras.backend.mean(keras.backend.square(y_pred[3:] - y_true[3:]), axis=-1)
        return err

    return loss

model = keras.Sequential([
    keras.layers.LSTM(256,
                      activation="relu",
                    #recurrent_activation="sigmoid",
                      use_bias=True,
                      input_shape=(look_back, 4)),
    keras.layers.Dense(4),
])
model.compile(optimizer="adam", loss=customloss(), metrics=["accuracy", "cosine_proximity"])


model.fit(train_generator, epochs=num_epochs, verbose=1, validation_data=val_generator, steps_per_epoch=len(train_generator))

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 176 steps, validate for 49 steps
Epoch 1/64
176/176 [==============================] - 2s 13ms/step - loss: 607.6630 - accuracy: 0.3709 - cosine_proximity: 0.9895 - val_loss: 65.5553 - val_accuracy: 0.0082 - val_cosine_proximity: 0.9994
Epoch 2/64
176/176 [==============================] - 1s 8ms/step - loss: 22.9906 - accuracy: 0.3392 - cosine_proximity: 0.9998 - val_loss: 37.8637 - val_accuracy: 0.5840 - val_cosine_proximity: 0.9998
Epoch 3/64
176/176 [==============================] - 1s 8ms/step - loss: 20.2733 - accuracy: 0.4036 - cosine_proximity: 0.9998 - val_loss: 56.5909 - val_accuracy: 0.0000e+00 - val_cosine_proximity: 0.9995
Epoch 4/64
176/176 [==============================] - 1s 8ms/step - loss: 25.4811 - accuracy: 0.3575 - cosine_proximity: 0.9998 - val_loss: 42.1358 - val_accuracy: 0.0082 - val_cosine_proximity: 0.9998
Epoch 5/64
176/176 [==============================] - 1s 8ms/step - loss: 18.3298 - accuracy:

In [38]:
prediction = model.predict(test_generator)

close_train_1 = close_train[0:, 3:].reshape((-1))
close_val_1 = close_val[0:, 3:].reshape((-1))
close_test_1 = close_test[0:, 3:].reshape((-1))
prediction_1 = prediction[0:, 3:].reshape((-1))

diff = close_test_1[look_back:] - prediction_1
avg_diff = np.mean(diff)
print(avg_diff)

def predict(num_prediction, before_lookback, model):
    if (before_lookback > 0):
        prediction_list = close_data[-look_back - before_lookback:-before_lookback]
    else:
        prediction_list = close_data[-look_back:]
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
#         print("Pred list", prediction_list)
        x = x.reshape((1, look_back, 4))
        out = model.predict(x)
#         print("Out", out)
        prediction_list = np.append(prediction_list, out, axis=0)
    prediction_list = prediction_list[look_back:]
        
    return prediction_list
    
def predict_dates(num_prediction, before_lookback):
    last_date = stonks.timestamp.values[-1 - before_lookback]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1+before_lookback).tolist()
    return prediction_dates

num_prediction = 4
before_lookback = 0
forecast = predict(num_prediction, before_lookback, model)[0:, 3:].reshape((-1))
forecast_dates = predict_dates(num_prediction, before_lookback)
prediction_1 = np.concatenate([[None] * look_back, prediction_1 + avg_diff])
trace1 = go.Scatter(
    x = np.concatenate([date_train, date_val]),
    y = np.concatenate([close_train_1, close_val_1]),
    mode = 'lines',
    name = 'Real Data - trained'
)
trace2 = go.Scatter(
    x = date_test,
    y = prediction_1,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = close_test_1,
    mode='lines',
    name = 'Real Data - untrained'
)
forecast = go.Scatter(
    x = forecast_dates,
    y = forecast + avg_diff,
    mode = 'lines+markers+text',
    texttemplate = "%{y:.2f}",
    name = 'Forecast'
)

layout = go.Layout(
    title = "Google Stock",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace1, trace2, trace3, forecast], layout=layout)
fig.show(renderer="browser")
# py.plot(fig, filename = 'basic-line', renderer="browser", auto_open=True)

0.7437268045486938


In [39]:
pio.write_html(fig, file='tracing_pred.html', auto_open=True)


In [34]:
model.save('good_model_bak_2.h5')

In [37]:
model = keras.models.load_model("good_model_bak_2.h5", custom_objects={'loss': customloss()})